In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
%load_ext autoreload
%autoreload 2

In [2]:
function_df = pd.read_csv('beta_lactamase/beta_lactamase_df.csv')

In [3]:
function_df.head()

,sequence,Description,Ligand,Mutational Tolerance,Units,Assay/Protocol
0,ASIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,M3A,NaN,0.0020,Unitless,Mutational Tolerance
1,CSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,M3C,NaN,0.0030,Unitless,Mutational Tolerance
2,DSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,M3D,NaN,0.0022,Unitless,Mutational Tolerance
3,ESIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,M3E,NaN,0.0024,Unitless,Mutational Tolerance
4,FSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,M3F,NaN,0.0057,Unitless,Mutational Tolerance


In [4]:
function_df['sequence_len'] = function_df['sequence'].apply(lambda seq: len(seq)) 
max_len = function_df['sequence_len'].max()

In [5]:
def sequence_to_aa(seq):
  return list(seq)

In [6]:
def get_sequence_aa(df):
  df['sequence_aa'] = df['sequence'].apply(lambda seq:sequence_to_aa(seq))
  df['length'] = df['sequence'].apply(lambda seq:len(seq))
  sequence_aa = np.array(df['sequence_aa'])
  return sequence_aa

In [7]:
sequence_aa = get_sequence_aa(function_df)

In [8]:
# import neccesary tools from Keras
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

Using TensorFlow backend.


In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequence_aa)
encoded = tokenizer.texts_to_sequences(sequence_aa)
vocab_len = len(tokenizer.word_index) + 1

In [10]:
X = np.array(encoded)
y = function_df['Mutational Tolerance']

from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.1)

In [11]:
from keras.layers import Input, Dense, Embedding, LSTM, Bidirectional, Dropout, BatchNormalization, Softmax, multiply, Lambda, Flatten, Activation, RepeatVector, Permute, LeakyReLU
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
from keras.optimizers import Adam
from keras.initializers import he_uniform, Constant

In [12]:
LSTM_size = 10
LSTM_dropout = 0.0
n_epochs = 2500
adam_lr = [0.001]
initializers = ['he_uniform']

In [15]:
# https://github.com/keras-team/keras/issues/4962
def get_model(initializer):
    input = Input(shape = [max_len])
    embedding = Embedding(input_dim = vocab_len, 
                          output_dim = 20, 
                          input_length = max_len, 
                          trainable = True)(input)

    activations = Bidirectional(LSTM(LSTM_size, return_sequences = True,
                       dropout = LSTM_dropout,
                       kernel_initializer = initializer))(embedding)

    attention = Dense(1, activation = 'tanh', kernel_initializer = initializer)(activations)
    attention = Flatten()(attention)
    attention = Activation('softmax', name = 'attention_activations')(attention)
    attention = RepeatVector(LSTM_size * 2)(attention)
    attention = Permute([2, 1])(attention)

    sequence_representation = multiply([activations, attention])
    sequence_representation = Lambda(lambda xin: K.sum(xin, axis = -2), output_shape = (LSTM_size * 2,))(sequence_representation)
    
    sequence_representation = Dense(64, kernel_initializer = initializer)(sequence_representation)
    sequence_representation = LeakyReLU()(sequence_representation)
    # sequence_representation = Dropout(0.2)(sequence_representation)
    sequence_representation = Dense(32, kernel_initializer = initializer)(sequence_representation)
    sequence_representation = LeakyReLU()(sequence_representation)
    # sequence_representation = Dropout(0.2)(sequence_representation)

    output = Dense(1, bias_initializer = Constant(0.5252))(sequence_representation)

    model = Model(inputs = [input], outputs = output)
    return model

In [ ]:
for i in range(len(adam_lr)):
    lr = adam_lr[i]
    for j in range(len(initializers)):
        initializer = initializers[j]
        optimizer = Adam(lr = lr)
        model = get_model(initializer)
        model.compile(optimizer = optimizer,
                     loss = 'mean_squared_error')
        # early_stopping = EarlyStopping(patience = 300)
        folder_path = 'beta_lactamase/model_and_checkpoints/' + f'LSTM size - {LSTM_size} epochs - {n_epochs} adam_lr {lr} initializer - {initializer}'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        checkpointing = ModelCheckpoint(folder_path + '/weights.{epoch:02d}-{val_loss:.5f}.hdf5', save_best_only = True)
        
        history = model.fit(X_train, y_train, validation_data = (X_val, y_val), 
                            batch_size = 256, epochs = n_epochs, verbose = 1,
                            callbacks = [checkpointing])

# save the model summary
# model_json = model.to_json()
# with open(f'LSTM units - {LSTM_units} epochs - {n_epochs} adam_lr {adam_lr} v3.json', 'w') as file:
#     file.write(model_json)
# get the Pearson score
# y_pred = model.predict(X_test)
# y_pred = np.reshape(y_pred, (1094,))

# from scipy.stats import pearsonr, spearmanr
# pearson_score = pearsonr(y_test, y_pred)
# print(f'Pearson correlation - {pearson_score}')
# spearman_score = spearmanr(y_test, y_pred)
# print(f'Spearman correlation - {spearman_score}')

# import pickle
# pickle.dump(pearson_score, open(f'LSTM units - {LSTM_units} epochs - {n_epochs} adam_lr {adam_lr} v3.p', 'wb'))

# visualize the training and validation loss curves
        plt.figure()
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epoch = np.arange(n_epochs)
        plt.plot(epoch, loss, label = 'loss')
        plt.plot(epoch, val_loss, label = 'val_loss')
        plt.title(f'LSTM size - {LSTM_size} LSTM dropout - {LSTM_dropout} epochs - {n_epochs} adam_lr {lr} initializer {initializer}')
        plt.legend()
        plt.savefig('beta_lactamase/' + f'LSTM size - {LSTM_size} LSTM dropout - {LSTM_dropout} epochs - {n_epochs} adam_lr {lr} initializer {initializer}.png')

Train on 3937 samples, validate on 438 samples
Epoch 1/2500
3937/3937 [==============================] - 8s 2ms/step - loss: 0.2024 - val_loss: 0.2035
Epoch 2/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2024 - val_loss: 0.2034
Epoch 3/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2022 - val_loss: 0.2034
Epoch 4/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2022 - val_loss: 0.2034
Epoch 5/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2022 - val_loss: 0.2034
Epoch 6/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2034
Epoch 7/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2022 - val_loss: 0.2034
Epoch 8/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2022 - val_loss: 0.2034
Epoch 9/2500
3937/3937 [==============================] - 5s 1ms/step - loss: 0.2022 - val_loss: 0.2035
Epoch 10/2500
393

3937/3937 [==============================] - 4s 1ms/step - loss: 0.2022 - val_loss: 0.2034
Epoch 79/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2033
Epoch 80/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2034
Epoch 81/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2022 - val_loss: 0.2033
Epoch 82/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2033
Epoch 83/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2034
Epoch 84/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2033
Epoch 85/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2022 - val_loss: 0.2034
Epoch 86/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2033
Epoch 87/2500
3937/3937 [==============================] - 4s 1ms/ste

3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2034
Epoch 156/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2033
Epoch 157/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2033
Epoch 158/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2033
Epoch 159/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2033
Epoch 160/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2033
Epoch 161/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2021 - val_loss: 0.2033
Epoch 162/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2020 - val_loss: 0.2033
Epoch 163/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2020 - val_loss: 0.2033
Epoch 164/2500
3937/3937 [==============================] - 4

3937/3937 [==============================] - 4s 1ms/step - loss: 0.1980 - val_loss: 0.1973
Epoch 233/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1980 - val_loss: 0.1990
Epoch 234/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1975 - val_loss: 0.1959
Epoch 235/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1992 - val_loss: 0.1963
Epoch 236/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1961 - val_loss: 0.1971
Epoch 237/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1927 - val_loss: 0.1943
Epoch 238/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1994 - val_loss: 0.2153
Epoch 239/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2010 - val_loss: 0.2044
Epoch 240/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.2007 - val_loss: 0.2014
Epoch 241/2500
3937/3937 [==============================] - 4

3937/3937 [==============================] - 4s 1ms/step - loss: 0.1812 - val_loss: 0.1836
Epoch 310/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1819 - val_loss: 0.1835
Epoch 311/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1812 - val_loss: 0.1849
Epoch 312/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1822 - val_loss: 0.1910
Epoch 313/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1800 - val_loss: 0.1845
Epoch 314/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1773 - val_loss: 0.1827
Epoch 315/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1766 - val_loss: 0.1890
Epoch 316/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1809 - val_loss: 0.1862
Epoch 317/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1810 - val_loss: 0.1833
Epoch 318/2500
3937/3937 [==============================] - 4

...

3937/3937 [==============================] - 4s 1ms/step - loss: 0.0892 - val_loss: 0.1553
Epoch 1687/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0874 - val_loss: 0.1552
Epoch 1688/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0867 - val_loss: 0.1570
Epoch 1689/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0943 - val_loss: 0.1528
Epoch 1690/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0891 - val_loss: 0.1544
Epoch 1691/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0898 - val_loss: 0.1736
Epoch 1692/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0900 - val_loss: 0.1521
Epoch 1693/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0893 - val_loss: 0.1544
Epoch 1694/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0886 - val_loss: 0.1647
Epoch 1695/2500
3937/3937 [==========================

3937/3937 [==============================] - 4s 1ms/step - loss: 0.0846 - val_loss: 0.1516
Epoch 1763/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0847 - val_loss: 0.1580
Epoch 1764/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0858 - val_loss: 0.1513
Epoch 1765/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0880 - val_loss: 0.1504
Epoch 1766/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0866 - val_loss: 0.1567
Epoch 1767/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0941 - val_loss: 0.1497
Epoch 1768/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0855 - val_loss: 0.1537
Epoch 1769/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0869 - val_loss: 0.1512
Epoch 1770/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0856 - val_loss: 0.1576
Epoch 1771/2500
3937/3937 [==========================

3937/3937 [==============================] - 4s 1ms/step - loss: 0.0891 - val_loss: 0.1508
Epoch 1839/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0840 - val_loss: 0.1517
Epoch 1840/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0840 - val_loss: 0.1513
Epoch 1841/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0890 - val_loss: 0.1825
Epoch 1842/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.1082 - val_loss: 0.1548
Epoch 1843/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0859 - val_loss: 0.1509
Epoch 1844/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0837 - val_loss: 0.1536
Epoch 1845/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0873 - val_loss: 0.1477
Epoch 1846/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0841 - val_loss: 0.1524
Epoch 1847/2500
3937/3937 [==========================

3937/3937 [==============================] - 4s 1ms/step - loss: 0.0828 - val_loss: 0.1514
Epoch 1915/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0851 - val_loss: 0.1539
Epoch 1916/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0814 - val_loss: 0.1562
Epoch 1917/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0795 - val_loss: 0.1543
Epoch 1918/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0808 - val_loss: 0.1592
Epoch 1919/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0866 - val_loss: 0.1542
Epoch 1920/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0819 - val_loss: 0.1521
Epoch 1921/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0822 - val_loss: 0.1514
Epoch 1922/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0792 - val_loss: 0.1527
Epoch 1923/2500
3937/3937 [==========================

3937/3937 [==============================] - 4s 1ms/step - loss: 0.0805 - val_loss: 0.1581
Epoch 1991/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0781 - val_loss: 0.1543
Epoch 1992/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0826 - val_loss: 0.1477
Epoch 1993/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0814 - val_loss: 0.1524
Epoch 1994/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0830 - val_loss: 0.1537
Epoch 1995/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0804 - val_loss: 0.1525
Epoch 1996/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0827 - val_loss: 0.1494
Epoch 1997/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0820 - val_loss: 0.1482
Epoch 1998/2500
3937/3937 [==============================] - 4s 1ms/step - loss: 0.0829 - val_loss: 0.1559
Epoch 1999/2500
3937/3937 [==========================

In [ ]:
model = get_model(initializers[0])
model.load_weights('beta_lactamase/model_and_checkpoints/LSTM size - 10 epochs - 2000 plus 1000 adam_lr 0.001 initializer - he_uniform/weights.38-0.11171.hdf5')

In [ ]:
# get the Pearson score
y_pred = model.predict(X_val)
y_pred = np.reshape(y_pred, (-1,))

from scipy.stats import pearsonr, spearmanr
pearson_score = pearsonr(y_val, y_pred)
print(f'Pearson correlation - {pearson_score}')
spearman_score = spearmanr(y_val, y_pred)
print(f'Spearman correlation - {spearman_score}')